# StressGait Analysis - Self-Reports

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.stats import StatsPipeline
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
bp.utils.file_handling.mkdirs([export_path, plot_path])

export_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, exclude_missing_data=True)

dataset

In [ ]:
condition_map = {"omc": "OMC", "control": "Control"}

In [ ]:
order = ["pre", "marker", "post"]
hue_order = ["Control", "OMC"]

In [ ]:
data = dataset.questionnaire
data = data.rename(index=condition_map)
data.head()

In [ ]:
quest_dict = {
    "SSSQ-pre": data.filter(like="Pre_SSSQ").columns,
    "SSSQ-marker": data.filter(like="PreMocap_SSSQ").columns,
    "SSSQ-post": data.filter(like="Post_SSSQ").columns,
    "PANAS-pre": data.filter(like="Pre_PANAS").columns,
    "PANAS-marker": data.filter(like="PreMocap_PANAS").columns,
    "PANAS-post": data.filter(like="Post_PANAS").columns,
}

In [ ]:
quest_data = compute_scores(
    data=data,
    quest_dict=quest_dict,
    quest_kwargs={"PANAS-pre": {"language": "german"}, "PANAS-post": {"language": "german"}},
)

quest_data.head()

In [ ]:
panas_data = wide_to_long(quest_data, quest_name="PANAS", levels=["subscale", "time"])
panas_data

In [ ]:
sssq_data = wide_to_long(quest_data, quest_name="SSSQ", levels=["subscale", "time"])
sssq_data = sssq_data.reindex(["Distress", "Total", "SelfEvaluation"], level="subscale")
sssq_data

## PANAS

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "mixed_anova")],
    params={
        "dv": "PANAS",
        "subject": "participant",
        "within": "time",
        "between": "condition",
        "groupby": "subscale",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(panas_data.drop("marker", level="time"))

# box_pairs, pvalues = pipeline.sig_brackets("posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within")

pipeline.display_results()

In [ ]:
panas_data_omc = panas_data.xs("OMC", level="condition")

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "PANAS",
        "subject": "participant",
        "within": "time",
        "groupby": "subscale",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(panas_data_omc)

box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within"
)

pipeline.display_results()

In [ ]:
fig, axs = plt.subplots(ncols=3)

for (key, val), ax in zip(panas_data_omc.groupby("subscale"), axs, strict=False):
    bp.plotting.feature_boxplot(
        data=val.reset_index(),
        x="time",
        y="PANAS",
        hue="time",
        order=order,
        ax=ax,
        stats_kwargs={"pairs": box_pairs[key], "pvalues": pvalues[key], "verbose": 0},
        palette=cmaps.fau_light,
    )
    ax.set_title(key)
    sns.swarmplot(data=val.reset_index(), x="time", y="PANAS", ax=ax, hue="time", palette=cmaps.fau[:3])

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(ncols=3)

handles, labels = None, None
for (key, val), ax in zip(panas_data.groupby("subscale"), axs, strict=False):
    sns.boxplot(
        data=val.reset_index(), x="time", y="PANAS", hue="condition", order=order, hue_order=["Control", "OMC"], ax=ax
    )
    ax.set_title(key)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend().remove()

fig.legend(handles, labels, ncols=2, loc="upper center")
fig.tight_layout(rect=(0, 0, 1, 0.95))

## SSSQ

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "SSSQ",
        "subject": "participant",
        "within": "time",
        "between": "condition",
        "groupby": "subscale",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(sssq_data.drop("marker", level="time"))

# box_pairs, pvalues = pipeline.sig_brackets("posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within")

pipeline.display_results(sig_only=False)

In [ ]:
sssq_data_omc = sssq_data.xs("OMC", level="condition")

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "SSSQ",
        "subject": "participant",
        "within": "time",
        "groupby": "subscale",
        "multicomp": {"method": "bonf", "levels": True},
    },
)

pipeline.apply(sssq_data_omc)

box_pairs, pvalues = pipeline.sig_brackets(
    "posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within"
)

pipeline.display_results()

In [ ]:
fig, axs = plt.subplots(ncols=3)

for (key, val), ax in zip(sssq_data_omc.groupby("subscale"), axs, strict=False):
    bp.plotting.feature_boxplot(
        data=val.reset_index(),
        x="time",
        y="SSSQ",
        hue="time",
        order=order,
        ax=ax,
        stats_kwargs={"pairs": box_pairs.get(key, []), "pvalues": pvalues.get(key, []), "verbose": 0},
        palette=cmaps.fau_light,
    )
    ax.set_title(key)
    sns.swarmplot(data=val.reset_index(), x="time", y="SSSQ", ax=ax, hue="time", palette=cmaps.fau[:3])

fig.tight_layout()

## VAS

In [ ]:
vas_data = data.filter(like="VAS")
vas_data = wide_to_long(vas_data, quest_name="VAS", levels="item")
vas_data.head()

In [ ]:
pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("test", "pairwise_tests")],
    params={
        "dv": "VAS",
        "between": "condition",
        "groupby": "item",
        "multicomp": {"method": "bonf", "levels": True},
        "parametric": False,
    },
)

pipeline.apply(vas_data)

# box_pairs, pvalues = pipeline.sig_brackets("posthoc", plot_type="single", x="subscale", subplots=True, stats_effect_type="within")

pipeline.display_results(sig_only=False)

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(data=vas_data.reset_index(), x="item", y="VAS", hue="condition", hue_order=hue_order, ax=ax)

fig.tight_layout()

## Plot for ISB Abstract

In [ ]:
from biopsykit.stats import StatsPipeline

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "pairwise_tests")], params={"dv": "PANAS", "subject": "participant", "within": "time"}
)

pipeline.apply(panas_data)

box_pairs, pvalues = pipeline.sig_brackets("test", stats_effect_type="within")

pipeline.display_results()

In [ ]:
fig, ax = plt.subplots()

bp.plotting.feature_boxplot(
    data=panas_data.reset_index(),
    x="time",
    y="PANAS",
    hue="time",
    show_legend=False,
    order=["pre", "marker", "post"],
    stats_kwargs={"pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    ax=ax,
)

fig.tight_layout()